<a href="https://colab.research.google.com/github/Evgeny-Artemov/Deep_Learning/blob/main/Educational-projects/Similar_Images/Embedding_for_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget 'https://data.caltech.edu/records/mzrjq-6wc02/files/caltech-101.zip'
!unzip '/content/caltech-101.zip'
!tar -xf '/content/caltech-101/101_ObjectCategories.tar.gz'

In [ ]:
from PIL import Image
import os
import cv2
from google.colab.patches import cv2_imshow
import torch
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
from torchvision import datasets
from  torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

In [ ]:
#Приводим изображение к "стандартному размеру"
def resize(tensor, h = 256, w = 256):
  return cv2.resize(tensor, [h, w], interpolation = cv2.INTER_CUBIC)

In [ ]:
path = '/content/101_ObjectCategories'

# Все изображения сохранены в отдельных каталогах "Название каталога(категория изображения)/img1"
# Расайзнем, переименнуем(в имени изображения содержится категория) и сохраним изображения в отдельный каталог

def reorcan_set_in_random_Set(path):
  list_dir = os.listdir(path)

  for i in list_dir:
    list_img = os.listdir(os.path.join(path, i))

    for j in list_img:
      img_patch = os.path.join(path, i, j)
      img_list = img_patch.split('/')
      if '.jpg' in img_patch:
        img = cv2.imread(img_patch)
        tensor = resize(img)
        cv2.imwrite('/content/Image/' + '_'.join(img_list[3:]) , tensor)
        
reorcan_set_in_random_Set(path)

In [ ]:
# Прогонять изображания будем через encoder ResNet18(можно использовать любой)
# Делаем на GPU, так как это в 6 раз бысрее для данной модели
# CPU - 12 min, GPU - 2 min

device = torch.device('cuda:0')
model = resnet18(weights=ResNet18_Weights).to(device)
path = '/content/Image'

class clossly_set(Dataset):

  def __init__(self, path, transforms = None):
    self.image_set = os.listdir(path)
    self.transforms = transforms

  def __len__(self):
    return len(self.image_set)  

  def __getitem__(self, index):

    if torch.is_tensor(index):
      index = index.tolist()

    path_image = self.image_set[index]
    image_name = path_image.split('.')[0]
    
    img = cv2.imread(os.path.join(path, path_image))
    img = resize(img)

    if self.transforms:
      img = self.transforms(img)

    # Возвращаем само изображение, переведенное в тензор,нормализованное и класс к которому относится это изображение 
    return img, image_name

tr = transforms.Compose([
         transforms.ToTensor(), 
         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])         
    ])


dataset = clossly_set(path = path, transforms = tr)

#Заводим контейнер, в который будем складывать эмбединги изображений, ролученные на слое 'avgpool'
layer = model._modules.get('avgpool')
outputs = []

#Функция для сбора эмбедингов
def copy_embed(m,i,o):
  o = o[0:, :, 0,0].detach().cpu().numpy()
  outputs.append(o)

# При прохождении слоя 'avgpool' - (layer) будет вызываться функции copy_embed
# которая будет "регестрировать" прямой проход
_ = layer.register_forward_hook(copy_embed)

model.eval()

dataloader = DataLoader(dataset, batch_size = 1)

#Каждому эмбедингу соответствует лэйбл, складируем их в "index"
index = []

for iter, (x,y) in enumerate(dataloader):
  x_gpu = x.to(device)
  index.append(y)
  _, =  model(x_gpu)

# {'название изображения':'его эмбединг'}
hash_emb = {}
for i in range(len(index)):
  hash_emb[index[i][0]] = outputs[i][0]

In [ ]:
embed_data_frame = pd.DataFrame(hash_emb)